In [1]:
!pip install cudaq -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.3/72.3 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.1/237.1 MB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install qgear-lightning==1.0.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.1 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from qiskit import QuantumCircuit
from qgear_lightning.core import qiskit_to_gateList, counts_cudaq_to_qiskit, circ_kernel
from qgear_lightning.io import write4_data_hdf5, read4_data_hdf5
import cudaq

In [ ]:
#qiskit circuit
n_qubits = 4
qc = QuantumCircuit(n_qubits)
qc.h(0)
qc.cx(0, 1)
qc.ry(np.pi/4, 2)
qc.u(np.pi/2, np.pi/4, np.pi/8, 3)
qc.measure_all()


In [ ]:
qcEL = [qc]  # List of circuits
numShots = 10000
verb = 0
backend = "qpp-cpu"

out_path = "out"
os.makedirs(out_path, exist_ok=True)
circ_name = "toy_example"
inpF = os.path.join(out_path, circ_name + ".gate_list.h5")


In [ ]:
# Convert to gate list
outD, md = qiskit_to_gateList(qcEL)
md["short_name"] = circ_name
write4_data_hdf5(outD, inpF, md)
# Load back
gateD, MD = read4_data_hdf5(inpF, verb)



In [ ]:
num_qubits=0

nc = len(gateD['circ_type'])
resL = [0] * nc
stateL = [0] * nc

for i in range(nc):
    num_qubit, num_gate = map(int, gateD['circ_type'][i])
    gate_type = list(map(int, gateD['gate_type'][i].flatten()))
    gate_param = list(map(float, gateD['gate_param'][i].flatten()))
    assert num_gate <= len(gate_param)
    prOn = num_qubit < 6 and i == 0 or verb > 1
    num_qubits = num_qubit
    print(cudaq.draw(circ_kernel, num_qubit, num_gate, gate_type, gate_param))

In [ ]:
@cudaq.kernel
def circ_kernel(qvector: cudaq.qvector, num_gate: int, gate_type: list[int], angles: list[float]):
    for i in range(num_gate):
        j = 3 * i
        gateId = gate_type[j]
        q0 = qvector[gate_type[j+1]]

        if gateId == 1:  # h
            h(q0)
        elif gateId == 2:  # ry
            ry(angles[j], q0)
        elif gateId == 3:  # rz
            rz(angles[j], q0)
        elif gateId == 4:  # cx
            q1 = qvector[gate_type[j + 2]]
            x.ctrl(q0, q1)
        elif gateId == 5:  # measure placeholder
            continue
        elif gateId == 6:  # cp
            q1 = qvector[gate_type[j + 2]]
            r1.ctrl(angles[j], q0, q1)
        elif gateId == 7:  # swap
            q1 = qvector[gate_type[j + 2]]
            swap(q0, q1)
        elif gateId == 8:  # u3
            theta = angles[j]
            phi = angles[j+1]
            lambda_ = angles[j+2]
            u3(theta, phi, lambda_, q0)

# create your own unitary
def decrementer(num_qubits):
    size = 2**num_qubits
    dec_matrix = np.zeros((size, size))
    for i in range(size):
        dec_matrix[i, (i + 1) % size] = 1
    return dec_matrix

# Create a custom register operation for the decrementer called DEC
cudaq.register_operation("DEC", decrementer(num_qubits))

@cudaq.kernel
def measured_circ_kernel(num_qubit: int, num_gate: int, gate_type: list[int], angles: list[float]):
    q = cudaq.qvector(num_qubit)
    circ_kernel(q, num_gate, gate_type, angles)
    DEC(q[0], q[1], q[2], q[3])
    mz(q)  # measure all qubits in Z-basis

# Example parameters
num_qubit = 4
num_gate = 2
gate_type = [1, 0, 0, 2, 1, 0]  # h on q0, ry on q1
gate_param = [0.0, 0.5, 0.0] * num_gate

counts = cudaq.sample(measured_circ_kernel, num_qubit, num_gate, gate_type, gate_param)
print(counts)
